In [13]:
import pandas as pd
import glob
import os

colors = pd.read_csv('data/team_colors.csv')

champs_glob = glob.glob(os.path.join('data/champs', "*.csv"))

df_from_each_file = (pd.read_csv(f) for f in champs_glob)
all_champs   = pd.concat(df_from_each_file, ignore_index=True)

data = colors.merge(all_champs, how = "right", on = ["Team", "League"])

In [14]:
winners = data[data["Wins"] > 0]
winners.head()

,Team,hex_Primary_Color,hex_Secondary_Color,League,wc_Primary_Name,wc_Secondary_Name,wa_Primary_Names,wa_Secondary_Names,wa_Primary_Name_1,wa_Primary_Name_2,wa_Secondary_Name_1,wa_Secondary_Name_2,Wins
2,Baltimore Ravens,#241773,#000000,NFL,midnightblue,black,"midnight blue,navy","black,dim gray",midnight blue,navy,black,dim gray,2
5,Chicago Bears,#0B162A,#C83803,NFL,black,firebrick,"midnight blue,steel blue","orange red,chocolate",midnight blue,steel blue,orange red,chocolate,1
8,Dallas Cowboys,#041E42,#869397,NFL,midnightblue,lightslategray,"navy,dark blue","light slate gray,slate gray",navy,dark blue,light slate gray,slate gray,5
9,Denver Broncos,#002244,#FB4F14,NFL,midnightblue,orangered,"navy,teal","orange red,chocolate",navy,teal,orange red,chocolate,3
11,Green Bay Packers,#203731,#FFB612,NFL,darkslategray,orange,"dark slate gray,cadet blue","orange,dark orange",dark slate gray,cadet blue,orange,dark orange,4


In [15]:
winners[winners["hex_Primary_Color"].isna()]

,Team,hex_Primary_Color,hex_Secondary_Color,League,wc_Primary_Name,wc_Secondary_Name,wa_Primary_Names,wa_Secondary_Names,wa_Primary_Name_1,wa_Primary_Name_2,wa_Secondary_Name_1,wa_Secondary_Name_2,Wins


In [18]:
#winners.groupby(["wa_Primary_Name_1"])["Wins"].sum().reset_index()
winners.groupby(["wc_Primary_Name"])["Wins"].sum().reset_index()

,wc_Primary_Name,Wins
0,black,104
1,blue,5
2,brown,50
3,crimson,150
4,darkblue,6
5,darkcyan,19
6,darkgreen,2
7,darkkhaki,18
8,darkred,5
9,darkslateblue,22
